## Extract, Transform and Load (ETL) Step

We are going to use a custom CNN for object identification.

### Setup

In [3]:
import os
import random
import time
from dataclasses import dataclass
from pathlib import Path
from enum import Enum
import shutil
from tqdm import tqdm
import tensorflow as tf
import keras
import math
import json
import numpy as np
from sklearn.utils import class_weight
from keras.applications import EfficientNetB3
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



ROOT_DIR = "datasets\\aircraft"
DATASET_DIR = "..\\dataset\\crop"

## Classes Mapping

We also need to provide a classes mapping, here is how we do that

In [4]:
CLASSES_RAW = [
    "a10",
    "a400m",
    "ag600",
    "ah64",
    "av8b",
    "an124",
    "an22",
    "an225",
    "an72",
    "b1",
    "b2",
    "b21",
    "b52",
    "be200",
    "c130",
    "c17",
    "c2",
    "c390",
    "c5",
    "ch47",
    "cl415",
    "e2",
    "e7",
    "ef2000",
    "f117",
    "f14",
    "f15",
    "f16",
    "f22",
    "f35",
    "f4",
    # IN DATA AS F18
    "f18",
    "h6",
    "j10",
    "j20",
    "jas39",
    "jf17",
    "jh7",
    "kc135",
    "kf21",
    "kj600",
    "ka27",
    "ka52",
    "mq9",
    "mi24",
    "mi26",
    "mi28",
    "mig29",
    "mig31",
    "mirage2000",
    "p3",
    "rq4",
    "rafale",
    "sr71",
    "su24",
    "su25",
    "su34",
    "su57",
    "tb001",
    "tb2",
    "tornado",
    "tu160",
    "tu22m",
    "tu95",
    "u2",
    "uh60",
    "us2",
    "v22",
    "vulcan",
    "wz7",
    "xb70",
    "y20",
    "yf23",
    "z19"
]

CLASSES = {model: i for i, model in enumerate(CLASSES_RAW)}

def get_class_id(class_str: str):
    return CLASSES[class_str.lower()]

print(CLASSES)

{'a10': 0, 'a400m': 1, 'ag600': 2, 'ah64': 3, 'av8b': 4, 'an124': 5, 'an22': 6, 'an225': 7, 'an72': 8, 'b1': 9, 'b2': 10, 'b21': 11, 'b52': 12, 'be200': 13, 'c130': 14, 'c17': 15, 'c2': 16, 'c390': 17, 'c5': 18, 'ch47': 19, 'cl415': 20, 'e2': 21, 'e7': 22, 'ef2000': 23, 'f117': 24, 'f14': 25, 'f15': 26, 'f16': 27, 'f22': 28, 'f35': 29, 'f4': 30, 'f18': 31, 'h6': 32, 'j10': 33, 'j20': 34, 'jas39': 35, 'jf17': 36, 'jh7': 37, 'kc135': 38, 'kf21': 39, 'kj600': 40, 'ka27': 41, 'ka52': 42, 'mq9': 43, 'mi24': 44, 'mi26': 45, 'mi28': 46, 'mig29': 47, 'mig31': 48, 'mirage2000': 49, 'p3': 50, 'rq4': 51, 'rafale': 52, 'sr71': 53, 'su24': 54, 'su25': 55, 'su34': 56, 'su57': 57, 'tb001': 58, 'tb2': 59, 'tornado': 60, 'tu160': 61, 'tu22m': 62, 'tu95': 63, 'u2': 64, 'uh60': 65, 'us2': 66, 'v22': 67, 'vulcan': 68, 'wz7': 69, 'xb70': 70, 'y20': 71, 'yf23': 72, 'z19': 73}


### Defining Dataset Constants

In [5]:
IMG_WIDTH = 256 # Keras default
IMG_HEIGHT = 256 # Keras default
IMG_CHANNELS = 3 # RGB

BATCH_SIZE = 32
EPOCHS = 10

### Extracting the Dataset

We need to first extract the x and y data from the dataset (image paths and labels).

### Splitting the Dataset

We need to split out dataset into three parts, train, validation and test.

We're going to use (80%, 10%, 10%) repartition for now.

In [6]:
def split_dataset(paths: list[str], classes: list[int], seed: int = None) -> tuple[list[str], list[int], list[str], list[int], list[str], list[int]]:
    x_train = []
    y_train = []
    x_val = []
    y_val = []
    x_test = []
    y_test = []

    if not seed:
        seed = time.time()

    random.seed(seed)

    for i in tqdm(range(len(paths))):
        rand = random.randint(1, 10)

        # Validation (10%)
        if rand == 9:
            x_val.append(paths[i])
            y_val.append(classes[i])

        # Test (10%)
        elif rand == 10:
            x_test.append(paths[i])
            y_test.append(classes[i])

        # Train (80%)
        else:
            x_train.append(paths[i])
            y_train.append(classes[i])

    return x_train, y_train, x_val, y_val, x_test, y_test

def extract_dataset(dataset_dir: str = DATASET_DIR, seed: int = None) -> None:
    aircraft_filepaths = []
    aircraft_classes = []
    for aircraft_dir in tqdm(os.listdir(dataset_dir)):
        aircraft_class = get_class_id(aircraft_dir)

        dir_path = os.path.join(dataset_dir, aircraft_dir)
        
        for aircraft_img in os.listdir(dir_path):
            aircraft_img_path = os.path.join(dir_path, aircraft_img)

            aircraft_filepaths.append(aircraft_img_path)
            aircraft_classes.append(aircraft_class)

    print(f"Found {len(aircraft_filepaths)} aircraft images")

    return aircraft_filepaths, aircraft_classes

paths, classes = extract_dataset()

x_train, y_train, x_val, y_val, x_test, y_test = split_dataset(paths, classes)

print(f"Train: {len(x_train)}")
print(f"Validation: {len(x_val)}")
print(f"Test: {len(x_test)}")

100%|██████████| 74/74 [00:00<00:00, 895.57it/s]


Found 31917 aircraft images


100%|██████████| 31917/31917 [00:00<00:00, 1386272.89it/s]

Train: 25464
Validation: 3234
Test: 3219


### Transforming the Dataset

We need to transform the dataset into a format that can be used by the model.

This includes

- Loading the images as tensors
- Resizing the images
- Normalizing the images (done later by the model)

### Lazy Loading and Tensorflow Datasets

We're going to use the `tf.data.Dataset` API to load the images lazily.

This is done to avoid loading all the images into RAM at once (10GB+ of images).

We also need to shuffle the dataset to avoid biasing the model since the images are ordered by class.

Notes:

- We're going to use a batch size of 32 for now
- We're going to use a prefetch buffer, automatically tuned by Tensorflow


In [7]:
class AircraftDataGenerator:
    def __init__(self, filepaths: list[str], classes: list[int], batch_size: int = BATCH_SIZE):
        self.filepaths = filepaths
        self.classes = classes
        self.batch_size = batch_size

    def load_image(self, filepath: str) -> tf.Tensor:
        image = tf.io.read_file(filepath)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])
        return image

    def create_dataset(self) -> tf.data.Dataset:
        dataset = tf.data.Dataset.from_tensor_slices((self.filepaths, self.classes))

        dataset = dataset.shuffle(buffer_size=len(self.filepaths), reshuffle_each_iteration=True)
        
        dataset = dataset.map(lambda x, y: (self.load_image(x), y), 
                            num_parallel_calls=tf.data.AUTOTUNE)
        
        
        dataset = dataset.batch(self.batch_size)

        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        return dataset
    
train_generator = AircraftDataGenerator(x_train, y_train)
val_generator = AircraftDataGenerator(x_val, y_val)
test_generator = AircraftDataGenerator(x_test, y_test)

train_dataset = train_generator.create_dataset()
val_dataset = val_generator.create_dataset()
test_dataset = test_generator.create_dataset()

### Building the Model Tests



In [8]:
def build_data_augmentation():
    """Creates a data augmentation sequential model."""
    return keras.Sequential(
        [
            keras.layers.RandomFlip("horizontal"),
            keras.layers.RandomRotation(0.1),
        ],
        name="data_augmentation"
    )

def build_base_model(include_data_augmentation=False):
    """Builds the base model with or without data augmentation."""
    layers = []
    
    if include_data_augmentation:
        data_augmentation = build_data_augmentation()
        layers.append(data_augmentation)
    
    base_model = EfficientNetB3(
        include_top=False,
        weights="imagenet",
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
        pooling="max",
    )
    layers.append(base_model)
    layers.append(keras.layers.BatchNormalization(name="batch_normalization"))
    layers.append(
        keras.layers.Dense(
            256, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01), name="dense_256"
        )
    )
    layers.append(keras.layers.Dropout(0.2, name="dropout_0.2"))
    layers.append(
        keras.layers.Dense(len(CLASSES), activation="softmax", name="output_layer")
    )
    
    model = keras.Sequential(layers, name="efficientnetb3_model")
    return model

def compile_model(model, optimizer='adam', learning_rate=0.001):
    """Compiles the model with specified optimizer and learning rate."""
    if optimizer.lower() == 'adamax':
        optimizer_instance = keras.optimizers.Adamax(learning_rate=learning_rate)
    else:
        optimizer_instance = keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(
        optimizer=optimizer_instance,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

def get_class_weights(y_train):
    """Calculates class weights for handling class imbalance."""
    y_train_np = np.array(y_train)
    class_weights_values = class_weight.compute_class_weight(
        "balanced", classes=np.unique(y_train_np), y=y_train_np
    )
    class_weights_dict = dict(enumerate(class_weights_values))
    return class_weights_dict

def get_callbacks():
    """Defines callbacks for training."""
    def lr_decay(epoch):
        return 0.001 * math.pow(0.666, epoch)
    
    lr_decay_callback = keras.callbacks.LearningRateScheduler(lr_decay, verbose=1)
    early_stop_callback = keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True, verbose=1
    )
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath="best_model_sofar.keras",
        monitor="val_loss",
        save_best_only=True,
        verbose=1,
    )
    return [lr_decay_callback, early_stop_callback, checkpoint_callback]

### Test Multiple Configurations

In [9]:

def train_basic():
    """Trains the basic model without any enhancements."""
    print("Training Basic Model...")
    model = build_base_model(include_data_augmentation=False)
    model = compile_model(model, optimizer='adam', learning_rate=0.001)
    model.summary()
    
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        verbose=1
    )
    
    model.evaluate(test_dataset)
    model.save("basic_model.keras")
    print("Basic Model saved")
    return history

def train_adamax():
    """Trains the model using the Adamax optimizer."""
    print("Training with Adamax Optimizer...")
    model = build_base_model(include_data_augmentation=False)
    model = compile_model(model, optimizer='adamax', learning_rate=0.001)
    model.summary()
    
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        verbose=1
    )
    
    model.evaluate(test_dataset)
    model.save("adamax_model.keras")
    print("Adamax Model saved")
    return history

def train_data_augmentation():
    """Trains the model with data augmentation."""
    print("Training with Data Augmentation...")
    model = build_base_model(include_data_augmentation=True)
    model = compile_model(model, optimizer='adam', learning_rate=0.001)
    model.summary()
    
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        verbose=1
    )
    
    model.evaluate(test_dataset)
    model.save("data_augmentation_model.keras")
    print("Data Augmentation Model saved")
    return history

def train_class_weights():
    """Trains the model with class weights to handle class imbalance."""
    print("Training with Class Weights...")
    model = build_base_model(include_data_augmentation=False)
    model = compile_model(model, optimizer='adam', learning_rate=0.001)
    model.summary()
    
    class_weights_dict = get_class_weights(y_train)
    
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        class_weight=class_weights_dict,
        verbose=1
    )
    
    model.evaluate(test_dataset)
    model.save("class_weights_model.keras")
    print("Class Weights Model saved")
    return history

def train_full_no_cb():
    """Trains the full model with all enhancements and callbacks."""
    print("Training Full Model with All Enhancements (No CallBacks)...")
    model = build_base_model(include_data_augmentation=True)
    model = compile_model(model, optimizer='adamax', learning_rate=0.001)
    model.summary()
    
    class_weights_dict = get_class_weights(y_train)
    
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        class_weight=class_weights_dict,
        verbose=1
    )
    
    model.evaluate(test_dataset)
    model.save("full_model_no_cb.keras")
    print("Full Model (No CallBacks) saved")
    return history

def train_full_cb():
    """Trains the full model with all enhancements and callbacks."""
    print("Training Full Model with All Enhancements (With CallBacks)...")
    model = build_base_model(include_data_augmentation=True)
    model = compile_model(model, optimizer='adamax', learning_rate=0.001)
    model.summary()
    
    class_weights_dict = get_class_weights(y_train)
    callbacks = get_callbacks()
    
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        class_weight=class_weights_dict,
        callbacks=callbacks,
        verbose=1
    )
    
    model.evaluate(test_dataset)
    model.save("full_model_cb.keras")
    print("Full Model (With CallBacks) saved")
    return history

def train_optimal():
    """Trains the full model with all enhancements and callbacks optimally."""
    print("Training Full Model Optimally (30 EPOCHS)")
    model = build_base_model(include_data_augmentation=True)
    model = compile_model(model, optimizer="adamax", learning_rate=0.001)
    model.summary()

    callbacks = get_callbacks()

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=30,
        callbacks=callbacks,
        verbose=1,
    )

    model.evaluate(test_dataset)
    model.save("full_model_optimal.keras")
    print("Full Model Optimal saved")
    return history

### And... Benchmark the Models

In [10]:
def main():
    """Main function that runs all training configurations and logs their performance."""
    training_functions = [
        train_basic,
        train_adamax,
        train_data_augmentation,
        train_class_weights,
        train_full_no_cb,
        train_full_cb,
        train_optimal,
    ]
    
    results = []
    
    for train_fn in training_functions:
        result = train_fn()
        result.history["Training Name"] = train_fn.__name__
        results.append(result)
    
    with open('training_histories.json', 'w') as f:
        json.dump([history.history for history in results], f)
    

### Running the Code and Training the Models

In [11]:
# Trained on Remote GPU from University
# main()

### Testing the Models


In [15]:
def evaluate_models():
    model_filenames = [
        "full_model_optimal.keras",
    ]

    for model_filename in model_filenames:
        model_path = os.path.join('training_weights', model_filename)
        
        model = tf.keras.models.load_model(model_path)
        
        y_true = []
        y_pred = []
        
        for images, labels in test_dataset:
            predictions = model.predict(images, verbose=0)
            predicted_labels = tf.argmax(predictions, axis=1).numpy()
            y_true.extend(labels.numpy())
            y_pred.extend(predicted_labels)
        
        cm = confusion_matrix(y_true, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(CLASSES.keys()))
        
        fig, ax = plt.subplots(figsize=(12, 12))
        disp.plot(cmap=plt.cm.Blues, ax=ax, xticks_rotation='vertical')
        plt.title(f'Confusion Matrix for {model_filename}')
        
        cm_filename = f"{model_filename}_confusion_matrix.png"
        cm_path = os.path.join('stats', cm_filename)
        plt.savefig(cm_path)
        plt.close()
        
        if model_filename == 'train_optimal':
            disp.plot(cmap=plt.cm.Blues)
            plt.title(f'Confusion Matrix for {model_filename}')
            plt.show()

In [ ]:
evaluate_models()

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 832ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 838ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 834ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 848ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 849ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 818ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 823ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 816ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 820ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 855ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 843ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 831ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 831ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 872ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 849ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 810ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 831ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 825ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 856ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 825ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 855ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 830ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 840ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 823ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

## Results Analysis

1. **Basic Model Optimizers:**
   - The baseline model (`train_basic`) starts with ~43% accuracy and reaches ~90% accuracy on the training set but validation accuracy plateaus at around ~80%.
   - The `train_adamax` run shows a significant improvement in both training and validation performance early on, reaching nearly 89%–90% validation accuracy by the end of training. This suggests that the Adamax optimizer is more suitable than the Adam optimizer (with their respective hyperparameters).

2. **Data Augmentation (`train_data_augmentation`):**
   - The run with data augmentation starts from a lower initial accuracy (completely normal), but the final validation accuracy is around 82%. Data augmentation normally generalizes the model better and reduces overfitting. The slight improvement compared to `train_basic` might become more pronounced if trained for more epochs.

3. **Class Weights (`train_class_weights`):**
   - Although there is a class imbalance, class weights seems to make training more difficult. The validation accuracy fluctuates widely and doesn’t steadily improve. Maybe it's not optimal with the current optimizer and learning rate.

4. **Full Model With No Callbacks (`train_full_no_cb`):**
   - This approach, which includes data augmentation, Adamax optimizer, and class weighting or some combination thereof, starts with a low initial accuracy (probably due to complexity and/or augmented data) but eventually reaches a high training accuracy (94%) and decent validation accuracy (~88%). This indicates your enhancements are working, but you might still benefit from a better learning rate schedule or more epochs.

5. **Full Model With Callbacks (`train_full_cb`):**
   - Using callbacks (learning rate scheduler, early stopping, checkpoints) plus enhancements leads to steady improvements. The final validation accuracy stabilizes around the mid-80s. The learning rate schedule seems effective, as the loss consistently decreases.

6. **Optimal Model (`train_optimal`):**
   - The optimal model configuration, Adamax trained on 30 EPOCHS with a learning rate scheduler, early stopping, and checkpoints, starts with a lower initial accuracy but reaches a high training accuracy (97%) and a validation accuracy of ~90%. This is a good result. Even though data augmentation slightly reduces accuracy, we keep it because real life images could vary significantly from dataset.

## Potential Improvements

1. **Longer Training or Fine-Tuning More Layers:**
   - EfficientNetB3 is powerful, but with limited experiment epochs (only 10), we might not be fully converging. We can increase experiment epochs to 20 or 30 to see if validation accuracy improves (takes too long to train).
   - Experiment with freezing / unfreezing the EfficientNetB3 layers.

2. **Hyperparameter Tuning:**
   - Test different learning rates. Values tested : 1e-3, 1e-4, 1e-5 on Adam and Adamax optimizers.
   - Test different decay rates for the learning rate scheduler.
   - Test different L2 regularization rates.
   - Test different batch sizes (32, 64, 128).

3. **More Aggressive Data Augmentation:**
   - Test different data augmentations (contrast, zoom, brightness) to improve generalization.
   - Simulating real world conditions (light, weather, lighting, etc.) could help the model generalize better.

4. **Class Balance and Class Weights:**
   - The dataset is quite unbalanced, so we could experiment with different class weights or other techniques to handle class imbalance.
   - However, it seems that class weights might not be the best approach for this dataset as it significantly affects training stability.

5. **Transfer Learning Strategies:**
   - Test using larger EfficientNet variant (B4 or later) to see if performance improves.
   - Test other architectures like ResNet, DenseNet, or Inception.
   - Test other pre-trained models like VGG, MobileNet, or Xception.

## Conclusions

Transfer learning was the way to go with this dataset.

Building a CNN from the ground up did not perform well despite lots of testing, it never got past 30% testing accuracy no matter what I tried.

YOLO and Transfer Learning with EfficientNetB3 were the best performing models, with EfficientNetB3 reaching 90% validation and testing accuracy with the optimal configuration.